In [1]:
# Установка дополнительных библиотек
%pip install plotly --upgrade -q
%pip install seaborn --upgrade -q

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings

# Настройки
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')  # Используем актуальный стиль seaborn
sns.set_theme(style="whitegrid")  # Инициализация seaborn
sns.set_palette('husl')

class WeatherDashboard:
    def __init__(self, api_key):
        self.API_KEY = api_key
        self.BASE_URL = 'http://api.openweathermap.org/data/2.5/forecast'
        self.weather_data = None
        self.city = None
        
    def fetch_data(self, city, units='metric', lang='ru'):
        """Получение данных о погоде с OpenWeatherMap API"""
        self.city = city
        params = {
            'q': city,
            'appid': self.API_KEY,
            'units': units,
            'lang': lang
        }
        
        try:
            response = requests.get(self.BASE_URL, params=params)
            response.raise_for_status()
            data = response.json()
            
            processed_data = []
            for forecast in data['list']:
                processed_data.append({
                    'datetime': pd.to_datetime(forecast['dt_txt']),
                    'temp': forecast['main']['temp'],
                    'feels_like': forecast['main']['feels_like'],
                    'temp_min': forecast['main']['temp_min'],
                    'temp_max': forecast['main']['temp_max'],
                    'humidity': forecast['main']['humidity'],
                    'pressure': forecast['main']['pressure'],
                    'wind_speed': forecast['wind']['speed'],
                    'wind_deg': forecast['wind']['deg'],
                    'clouds': forecast['clouds']['all'],
                    'weather': forecast['weather'][0]['main'],
                    'weather_desc': forecast['weather'][0]['description'],
                    'weather_icon': forecast['weather'][0]['icon']
                })
            
            self.weather_data = pd.DataFrame(processed_data)
            self.weather_data['date'] = self.weather_data['datetime'].dt.date
            self.weather_data['hour'] = self.weather_data['datetime'].dt.hour
            self.weather_data['day_of_week'] = self.weather_data['datetime'].dt.day_name()
            
            return self.weather_data
        
        except Exception as e:
            print(f"Ошибка: {e}")
            return None
    
    def create_dashboard(self, days=7):
        """Создание интерактивной панели в Colab"""
        if self.weather_data is None:
            print("Сначала загрузите данные с помощью fetch_data()")
            return
        
        # Фильтрация данных
        end_date = datetime.now() + timedelta(days=days)
        data = self.weather_data[self.weather_data['datetime'] <= end_date]
        
        # Стилизация HTML
        display(HTML(f"""
        <style>
            .weather-header {{
                background: linear-gradient(135deg, #6e8efb, #a777e3);
                color: white;
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 20px;
            }}
            .weather-card {{
                background: white;
                border-radius: 10px;
                padding: 15px;
                margin-bottom: 20px;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            }}
            .weather-title {{
                color: #4a4a4a;
                border-bottom: 2px solid #f0f0f0;
                padding-bottom: 10px;
                margin-bottom: 15px;
            }}
        </style>
        
        <div class='weather-header'>
            <h1>Погодный дашборд: {self.city}</h1>
            <h3>Период: {data['datetime'].min().strftime('%d.%m')} - {data['datetime'].max().strftime('%d.%m.%Y')}</h3>
        </div>
        """))
        
        # Текущая погода
        current = data.iloc[0]
        display(HTML(f"""
        <div class='weather-card'>
            <h2 class='weather-title'>Текущая погода</h2>
            <div style="display: flex; justify-content: space-between;">
                <div style="flex: 1;">
                    <h3 style="margin-top: 0;">{current['weather_desc'].capitalize()}</h3>
                    <p><b>Температура:</b> {current['temp']}°C (ощущается как {current['feels_like']}°C)</p>
                    <p><b>Влажность:</b> {current['humidity']}%</p>
                    <p><b>Давление:</b> {current['pressure']} hPa</p>
                    <p><b>Ветер:</b> {current['wind_speed']} м/с</p>
                </div>
                <div style="flex: 1; text-align: center;">
                    <img src="http://openweathermap.org/img/wn/{current['weather_icon']}@2x.png" 
                         style="width: 100px; height: 100px;">
                    <h2>{current['temp']}°C</h2>
                </div>
            </div>
        </div>
        """))
        
        # Интерактивный выбор дней
        day_slider = widgets.IntSlider(
            value=days,
            min=1,
            max=14,
            step=1,
            description='Дней:',
            continuous_update=False
        )
        
        widgets.interact(lambda days: self.update_plots(days), days=day_slider)
        
        # Первоначальное построение графиков
        self.update_plots(days)
    
    def update_plots(self, days):
        """Обновление графиков при изменении параметров"""
        if self.weather_data is None:
            return
        
        end_date = datetime.now() + timedelta(days=days)
        data = self.weather_data[self.weather_data['datetime'] <= end_date]
        
        # График температуры (Matplotlib)
        plt.figure(figsize=(12, 5))
        plt.plot(data['datetime'], data['temp'], label='Температура', color='#FF6B6B')
        plt.plot(data['datetime'], data['feels_like'], label='Ощущается как', linestyle='--', color='#4ECDC4')
        plt.fill_between(data['datetime'], data['temp_min'], data['temp_max'], color='#FFE66D', alpha=0.3)
        
        plt.title(f'Прогноз температуры на {days} дней', pad=15)
        plt.xlabel('Дата и время')
        plt.ylabel('Температура (°C)')
        plt.xticks(rotation=45)
        plt.grid(alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Интерактивный график Plotly
        fig = px.line(data, x='datetime', y=['temp', 'feels_like'],
                     title=f'Интерактивный прогноз температуры',
                     labels={'value': 'Температура (°C)', 'datetime': 'Дата и время'},
                     hover_data=['humidity', 'wind_speed', 'weather_desc'])
        
        fig.update_layout(
            hovermode='x unified',
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1д", step="day", stepmode="backward"),
                        dict(count=3, label="3д", step="day", stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(visible=True)
            )
        )
        fig.show()
        
        # Тепловая карта по часам
        pivot = data.pivot_table(values='temp', index='hour', columns='date', aggfunc='mean')
        
        plt.figure(figsize=(12, 5))
        sns.heatmap(pivot, cmap='coolwarm', annot=True, fmt=".1f", linewidths=.5)
        plt.title('Температура по дням и часам')
        plt.xlabel('Дата')
        plt.ylabel('Час дня')
        plt.show()
        
        # Круговая диаграмма погодных условий
        weather_counts = data['weather_desc'].value_counts()
        
        fig = px.pie(weather_counts, 
                    values=weather_counts.values, 
                    names=weather_counts.index,
                    title='Распределение погодных условий',
                    hole=0.3)
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.show()

# Инициализация дашборда
dashboard = WeatherDashboard(api_key='1718a77a63792e8d46952158940f26a0')

# Виджет для выбора города
city_input = widgets.Text(
    value='Москва',
    placeholder='Введите город',
    description='Город:',
    disabled=False
)

# Кнопка для загрузки данных
load_button = widgets.Button(
    description='Загрузить данные',
    button_style='success',
    tooltip='Нажмите для загрузки данных'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"Загружаем данные для {city_input.value}...")
        dashboard.fetch_data(city_input.value)
        dashboard.create_dashboard(days=7)

load_button.on_click(on_button_clicked)

# Отображение элементов управления
display(widgets.HBox([city_input, load_button]))
display(output)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Output()